In [ ]:
import  numpy as np
import pandas as pd

In [ ]:
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer

In [ ]:
import statsmodels.api as sm
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk.util import ngrams
from collections import Counter
from gensim.models import word2vec

In [ ]:
data = pd.read_csv('../input/train_file.csv')
test = pd.read_csv('../input/test_file.csv')

In [ ]:
data.shape

In [ ]:
data["MaterialType"] = data['MaterialType'].str.replace('[^\w\s]','')

In [ ]:
data.drop(['UsageClass','CheckoutType','CheckoutYear','CheckoutMonth','ID'],inplace=True,axis=1)
test.drop(['UsageClass','CheckoutType','CheckoutYear','CheckoutMonth'],inplace=True,axis=1)

In [ ]:
y = data["MaterialType"]

In [ ]:
data = data.apply(lambda x: x.astype(str).str.lower())
test = test.apply(lambda x: x.astype(str).str.lower())

In [ ]:
data["Title"] = data['Title'].str.replace('[^\w\s]','')
test["Title"] = test['Title'].str.replace('[^\w\s]','')
data["Creator"] = data['Creator'].str.replace('[^\w\s]','')
data["Subjects"] = data['Subjects'].str.replace('[^\w\s]','')
##data["Subjects"] = data['Subjects'].str.replace('[0-9]','')
data["Publisher"] = data['Publisher'].str.replace('[^\w\s]','')
data["PublicationYear"] = data['PublicationYear'].str.replace('[^\w\s]','')
data["PublicationYear"] = data['PublicationYear'].str.replace('[cp]','')
data["Creator"] = data['Creator'].str.replace('[0-9]','')
test["Subjects"] = test['Subjects'].str.replace('[^\w\s]','')
##test["Subjects"] = test['Subjects'].str.replace('[0-9]','')

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords

In [ ]:
sw = stopwords.words('english')

In [ ]:
def stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    return " ".join(text)

In [ ]:
data['Title'] = data['Title'].apply(stopwords)
data['Subjects'] = data['Subjects'].apply(stopwords)

test['Subjects'] = test['Subjects'].apply(stopwords)

In [ ]:
print(data.nunique())

In [ ]:
#from textblob import TextBlob
#data['Subjects'].apply(lambda x: str(TextBlob(x).correct()))

In [ ]:
#print(data.nunique())

In [ ]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = PorterStemmer()

In [ ]:
def pre(data):
    word = lemmatizer.lemmatize(data) 
    return word

In [ ]:
data['Title'] = data['Title'].apply(pre)
data['Subjects'] = data['Subjects'].apply(pre)
test['Subjects'] = test['Subjects'].apply(pre)
test['Title'] = test['Title'].apply(pre)

In [ ]:
final_title = data['Title']

In [ ]:
temp =[]
snow = nltk.stem.SnowballStemmer('english')
for sentence in data['Title']:
    sentence = sentence.lower()                 # Converting to lowercase
    cleanr = re.compile('<.*?>')
    sentence = re.sub(cleanr, ' ', sentence)        #Removing HTML tags
    sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence)        #Removing Punctuations
    
    words = [snow.stem(word) for word in sentence.split()]   # Stemming and removing stopwords
    temp.append(words)
    
data['Title'] = temp 

temp =[]
snow = nltk.stem.SnowballStemmer('english')
for sentence in data['Subjects']:
    sentence = sentence.lower()                 # Converting to lowercase
    cleanr = re.compile('<.*?>')
    sentence = re.sub(cleanr, ' ', sentence)        #Removing HTML tags
    sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence)        #Removing Punctuations
    
    words = [snow.stem(word) for word in sentence.split()]   # Stemming and removing stopwords
    temp.append(words)
    
data['Subjects'] = temp 

In [ ]:
sent = []
for row in data['Title']:
    sequ = ''
    for word in row:
        sequ = sequ + ' ' + word
    sent.append(sequ)

data['Title'] = sent
data

sent = []
for row in data['Subjects']:
    sequ = ''
    for word in row:
        sequ = sequ + ' ' + word
    sent.append(sequ)

data['Subjects'] = sent
data

In [ ]:
print(data.nunique())

In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
data["Subjects"] = data['Subjects'].replace('nan','')

In [ ]:
data["Final"] = data["Title"].map(str) + ' ' + data["Subjects"] 
test["Final"] = test["Title"].map(str) + ' ' + test["Subjects"]

In [ ]:
print(data.nunique())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
from sklearn.preprocessing import LabelEncoder

columns = ['MaterialType']

#def encoder(df):
    for col in columns:
        label_encoder = LabelEncoder()
        label_encoder.fit(df[col])
        df[col] = label_encoder.transform(df[col])
    return df

data = encoder(data)
test = encoder(data)

In [ ]:
x = data['Final']
w = test['Final']

In [ ]:
vectorizer = CountVectorizer(max_features=5000,ngram_range=(1,2))
vectorizer.fit(x)

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(w)

In [ ]:
W = vectorizer.transform(w)

In [ ]:
X = vectorizer.transform(x)
print('Shape of Sparse Matrix: ', X.shape)
print('Amount of Non-Zero occurrences: ', X.nnz)
# Percentage of non-zero values
density = (100.0 * X.nnz / (X.shape[0] * X.shape[1]))
print('Density: {}'.format((density)))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=100)
rfc = RandomForestClassifier(min_samples_split = 10, n_estimators = 100, n_jobs = -1)
rfc.fit(X_train, y_train)

# Testing the model
predictions = rfc.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
y_pred = rfc.predict(X_test)

In [ ]:
from sklearn import metrics
print("lgbms Accuracy: ", metrics.accuracy_score(y_test, y_pred))

In [ ]:
pred = rfc.predict(W)

In [ ]:
submission = pd.DataFrame({
        "ID": test["ID"],
        "MaterialType": pred})

In [ ]:
submission.to_csv('submission26.csv', index=False)